In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet, MobileNetv2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
class VeggieVision:
    def __init__(self, base_model, input_shape=(224, 224, 3)):
        # Set parameters
        self.input_shape = input_shape
        self.base_model = base_model
        self.model = None
        self.datagen = None

        # Build the model
        self.build_model()

    def build_model(self):
        # Freeze the base model layers initially
        self.base_model.trainable = False

       # 2. Build the Model
        self.model = models.Sequential()

        # Add the base model (MobileNetV2)
        self.model.add(self.base_model)

        self.model.add(layers.Flatten())

        # Add an additional Dense layer with 128 neurons and ReLU activation
        self.model.add(layers.Dense(1024, activation='relu'))


        self.model.add(layers.Dense(256, activation = "relu"))


        #   Final Dense layer with a single neuron for regression (predicting weight)
        self.model.add(layers.Dense(1, activation='linear'))
        

    def augment_data(self, X_train):
        # 5. Data Augmentation
        # Create an ImageDataGenerator for real-time data augmentation
        self.datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=180,         # Rotate images by up to 90 degrees
            width_shift_range=0.1,     # Shift image horizontally by up to 20% of the width
            height_shift_range=0.1,    # Shift image vertically by up to 20% of the height
            shear_range=0.1,           # Shear the image by up to 10%
            zoom_range=0.1,            # Zoom into the image by up to 20%
            horizontal_flip=True,      # Randomly flip images horizontally
            fill_mode='nearest'        # Fill in missing pixels with the nearest filled value
        )

        # Fit the ImageDataGenerator on the training data
        self.datagen.fit(X_train)

    def fit(self, X, y, batch_size=32, epochs=20, learning_rate=1e-4, validation_split=0.2, fine_tune=False):
        # 4. Split Data into Training and Validation Sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)

        # Augment the data
        self.augment_data(X_train)

        # Compile the model with the provided learning rate
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # 6. Train the Model
        # Use the data generator to augment and train the model
        history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                 validation_data=(X_val, y_val), 
                                 epochs=epochs)

        # Evaluate the model on the validation set
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after initial training: {test_mae}')

        # 8. Fine-tuning the model (if requested)
        if fine_tune:
            self.fine_tune(X_train, y_train, X_val, y_val, batch_size, epochs, learning_rate)

        return history

    def fine_tune(self, X_train, y_train, X_val, y_val, batch_size=32, epochs=10, fine_tune_learning_rate=1e-5):
        # Unfreeze the base MobileNetV2 model's layers
        self.base_model.trainable = True

        # Compile with a lower learning rate for fine-tuning
        self.model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # Fine-tune the model with augmented data
        fine_tune_history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                           validation_data=(X_val, y_val), 
                                           epochs=epochs)

        # Evaluate after fine-tuning
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after fine-tuning: {test_mae}')

        return fine_tune_history
    
    def evaluate(self, X, y):
        test_loss, test_mae = self.model.evaluate(X, y)
        print(f'MAE: {test_mae}')
        return test_loss, test_mae


In [4]:
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

In [13]:
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
veggie_eff = VeggieVision(base_model)


In [14]:
veggie_eff.fit(X, y, learning_rate = 1e-3)

Epoch 1/20
19/19 [==============================] - 33s 1s/step - loss: 22888.8633 - mae: 87.8497 - val_loss: 5630.7383 - val_mae: 66.1100
Epoch 2/20
19/19 [==============================] - 28s 1s/step - loss: 19469.5078 - mae: 77.9234 - val_loss: 6200.2412 - val_mae: 71.5822
Epoch 3/20
19/19 [==============================] - 29s 2s/step - loss: 19741.6172 - mae: 80.2633 - val_loss: 9207.1934 - val_mae: 85.0882
Epoch 4/20
19/19 [==============================] - 27s 1s/step - loss: 19710.7500 - mae: 78.2040 - val_loss: 6177.6377 - val_mae: 71.4565
Epoch 5/20
19/19 [==============================] - 28s 1s/step - loss: 19167.9375 - mae: 76.8420 - val_loss: 5638.6685 - val_mae: 66.7439
Epoch 6/20
19/19 [==============================] - 27s 1s/step - loss: 19089.9785 - mae: 75.9382 - val_loss: 5754.7930 - val_mae: 68.1551
Epoch 7/20
19/19 [==============================] - 26s 1s/step - loss: 19127.2656 - mae: 77.9496 - val_loss: 7117.5093 - val_mae: 76.4503
Epoch 8/20
19/19 [=========

KeyboardInterrupt: 

In [11]:
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large
base_model = MobileNetV3Large(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_v3 = VeggieVision(base_model)

In [12]:
veggie_v3.fit(X, y, learning_rate = 1e-3)

Epoch 1/20
19/19 [==============================] - 21s 948ms/step - loss: 20550.2500 - mae: 78.5622 - val_loss: 8868.7998 - val_mae: 80.2433
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 17755.7422 - mae: 71.3816

KeyboardInterrupt: 

In [47]:
from tensorflow.keras.applications import NASNetMobile
base_model = NASNetMobile(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggiemNAS = VeggieVision(base_model)

In [48]:
veggiemNAS.fit(X, y, learning_rate = 1e-6, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 41s 2s/step - loss: 34852.0703 - mae: 125.5409 - val_loss: 17441.1914 - val_mae: 108.8174
Epoch 2/20
19/19 [==============================] - 23s 1s/step - loss: 34059.1328 - mae: 122.3178 - val_loss: 16769.9238 - val_mae: 105.6850
Epoch 3/20
19/19 [==============================] - 24s 1s/step - loss: 33261.0273 - mae: 119.0830 - val_loss: 16081.8467 - val_mae: 102.4314
Epoch 4/20
19/19 [==============================] - ETA: 0s - loss: 32428.1191 - mae: 115.5845

KeyboardInterrupt: 

In [38]:
from tensorflow.keras.applications import MobileNet

# Load MobileNetV1 model pre-trained on ImageNet without the top classification layer
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggiemnv1 = VeggieVision(base_model)

17225924/17225924 [==============================] - 1s 0us/step


In [43]:
veggiemnv1.fit(X, y, learning_rate = 1e-6, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 17s 805ms/step - loss: 4417.5054 - mae: 19.6942 - val_loss: 614.0629 - val_mae: 16.8979
Epoch 2/20
19/19 [==============================] - 15s 780ms/step - loss: 6123.5176 - mae: 21.8302 - val_loss: 613.6417 - val_mae: 16.8940
Epoch 3/20
 1/19 [>.............................] - ETA: 16s - loss: 640.8345 - mae: 18.8356

KeyboardInterrupt: 

In [44]:
veggiemnv1.evaluate(X, y)

23/23 [==============================] - 10s 431ms/step - loss: 5019.7197 - mae: 20.3476
MAE: 20.347633361816406


(5019.7197265625, 20.347633361816406)

In [33]:
from tensorflow.keras.applications import EfficientNetB0

# Load EfficientNetB0 model pre-trained on ImageNet without the top classification layer
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggieE = VeggieVision(base_model)

In [34]:
veggieE.fit(X, y, learning_rate = 1e-3)

Epoch 1/20
19/19 [==============================] - 33s 1s/step - loss: 23441.6875 - mae: 89.7986 - val_loss: 9008.8477 - val_mae: 84.4435
Epoch 2/20
19/19 [==============================] - 26s 1s/step - loss: 19814.6641 - mae: 80.7464 - val_loss: 6243.9473 - val_mae: 71.8656
Epoch 3/20
19/19 [==============================] - ETA: 0s - loss: 19858.0137 - mae: 80.7530

KeyboardInterrupt: 

In [28]:
from tensorflow.keras.applications import DenseNet121

# Load DenseNet121 model pre-trained on ImageNet without the top classification layer
base_model = DenseNet121(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggieD = VeggieVision(base_model)

29084464/29084464 [==============================] - 1s 0us/step


In [31]:
veggieD.fit(X, y, learning_rate= 1e-5, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 54s 3s/step - loss: 10343.1104 - mae: 27.7153 - val_loss: 1011.5848 - val_mae: 24.1695
Epoch 2/20
19/19 [==============================] - 42s 2s/step - loss: 9528.8320 - mae: 26.8720 - val_loss: 1014.2749 - val_mae: 24.2335
Epoch 3/20
19/19 [==============================] - 42s 2s/step - loss: 9679.7617 - mae: 26.1666 - val_loss: 1009.5109 - val_mae: 24.0802
Epoch 4/20
19/19 [==============================] - 42s 2s/step - loss: 10207.9736 - mae: 26.1700 - val_loss: 1014.8134 - val_mae: 24.1771
Epoch 5/20
19/19 [==============================] - 42s 2s/step - loss: 9792.1250 - mae: 26.2294 - val_loss: 1007.2093 - val_mae: 24.0490
Epoch 6/20
16/19 [========================>.....] - ETA: 5s - loss: 11441.0908 - mae: 27.3599

KeyboardInterrupt: 

In [32]:
veggieD.evaluate(X, y)

23/23 [==============================] - 40s 2s/step - loss: 7891.6104 - mae: 27.2399
MAE: 27.2398681640625


(7891.6103515625, 27.2398681640625)

In [24]:
from tensorflow.keras.applications import Xception

# Initialize ResNet50 as the base model
base_model = Xception(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Create an instance of the VeggieVision class with the ResNet50 base model
veggieX = VeggieVision(base_model)

In [27]:
veggieX.fit(X, y, learning_rate= 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 45s 2s/step - loss: 11290.1572 - mae: 28.5359 - val_loss: 1524.3478 - val_mae: 26.6794
Epoch 2/20
19/19 [==============================] - 45s 2s/step - loss: 11258.8066 - mae: 30.6203 - val_loss: 1578.4894 - val_mae: 27.0292
Epoch 3/20
19/19 [==============================] - 40s 2s/step - loss: 11171.1006 - mae: 29.7092 - val_loss: 1536.6724 - val_mae: 26.7164
Epoch 4/20
19/19 [==============================] - 38s 2s/step - loss: 11737.9854 - mae: 30.5062 - val_loss: 1504.1456 - val_mae: 26.4793
Epoch 5/20
18/19 [===========================>..] - ETA: 1s - loss: 11250.7051 - mae: 29.0813

KeyboardInterrupt: 

In [14]:
from tensorflow.keras.applications import ResNet50

# Initialize ResNet50 as the base model
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Create an instance of the VeggieVision class with the ResNet50 base model
veggieres_one = VeggieVision(base_model)

In [15]:
veggieres_one.fit(X, y, learning_rate= 1e-3)

Epoch 1/20
19/19 [==============================] - 42s 2s/step - loss: 23809.1855 - mae: 90.3123 - val_loss: 6025.6699 - val_mae: 69.8317
Epoch 2/20
19/19 [==============================] - 35s 2s/step - loss: 17268.3105 - mae: 67.7113 - val_loss: 4284.1226 - val_mae: 55.9484
Epoch 3/20
19/19 [==============================] - 36s 2s/step - loss: 16314.3643 - mae: 60.8248 - val_loss: 4147.2837 - val_mae: 52.6011
Epoch 4/20
19/19 [==============================] - 36s 2s/step - loss: 16039.0225 - mae: 58.4353 - val_loss: 4703.8105 - val_mae: 57.6556
Epoch 5/20
19/19 [==============================] - 35s 2s/step - loss: 15402.2881 - mae: 59.1102 - val_loss: 4337.2476 - val_mae: 51.0180
Epoch 6/20


KeyboardInterrupt: 

In [6]:
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggiev_one = VeggieVision(base_model)

In [7]:
veggiev_one.fit(X,y, learning_rate = 1e-3)

Epoch 1/20
19/19 [==============================] - 52s 3s/step - loss: 34625.1562 - mae: 124.7175 - val_loss: 17481.6094 - val_mae: 109.0600
Epoch 2/20
19/19 [==============================] - 52s 3s/step - loss: 34236.5547 - mae: 123.1459 - val_loss: 17143.4980 - val_mae: 107.5160
Epoch 3/20
19/19 [==============================] - 54s 3s/step - loss: 33839.9492 - mae: 121.4883 - val_loss: 16813.0020 - val_mae: 105.9847
Epoch 4/20
12/19 [=================>............] - ETA: 16s - loss: 20359.4180 - mae: 115.1788

KeyboardInterrupt: 

In [11]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2 = VeggieVision(base_model)

In [12]:
veggie_mnv2.fit(X, y, learning_rate= 1e-3)

Epoch 1/20
19/19 [==============================] - 18s 786ms/step - loss: 33877.2656 - mae: 121.4344 - val_loss: 16145.1602 - val_mae: 102.6070
Epoch 2/20
19/19 [==============================] - 14s 715ms/step - loss: 31732.4824 - mae: 112.9834 - val_loss: 14555.7451 - val_mae: 94.7397
Epoch 3/20
19/19 [==============================] - 14s 712ms/step - loss: 29926.3008 - mae: 105.3934 - val_loss: 13121.5430 - val_mae: 87.6218
Epoch 4/20
19/19 [==============================] - ETA: 0s - loss: 28387.4004 - mae: 98.8951

KeyboardInterrupt: 